#Wrangling Subway Data 

##Basic Exploration of Weather Data 

In [1]:
import pandas
import pandasql
import os

In [2]:
# save path to weather data
path = os.path.realpath('data')
weather_data = path + '/weather_underground.csv'

###Number of Rainy Days 

In [3]:
def num_rainy_days(filename):
    
    weather_data = pandas.read_csv(filename)
    
    q = """
    SELECT
    COUNT(*)
    FROM 
    weather_data
    WHERE
    cast(rain as integer) > 0
    """
    
    #Execute your SQL command against the pandas frame
    rainy_days = pandasql.sqldf(q.lower(), locals())
    return rainy_days

In [4]:
num_rainy_days(path + "/weather_underground.csv")

,count(*)
0,10


In [5]:

data = pandas.read_csv(path + "/weather_underground.csv", nrows = 1)
print data


         date  maxpressurem  maxdewptm  maxpressurei  maxdewpti  \
0  2011-05-01          1026          6         30.31         42   

   since1julheatingdegreedaysnormal  heatingdegreedaysnormal  \
0                              4646                        8   

   since1sepcoolingdegreedaysnormal  hail  since1julsnowfallm    ...      \
0                               NaN     0              157.23    ...       

   precipi  snowfalli  since1jancoolingdegreedaysnormal  precipm  snowfallm  \
0        0          0                                13        0          0   

   thunder  monthtodateheatingdegreedays  meantempi  maxvism  meantempm  
0        0                             5         60       16         16  

[1 rows x 70 columns]


###Temp on Foggy and Non-Foggy Days 

In [6]:
def max_temp_aggregate_by_fog(filename):

    weather_data = pandas.read_csv(filename)

    q = """
    SELECT
    fog, cast(maxtempi as integer)
    FROM
    weather_data
    GROUP BY
    fog
    """
    
    #Execute your SQL command against the pandas frame
    foggy_days = pandasql.sqldf(q.lower(), locals())
    return foggy_days

In [7]:
max_temp_aggregate_by_fog(weather_data)

,fog,cast(maxtempi as integer)
0,0,86
1,1,81


###Mean Temp on Weekends 

In [8]:
def avg_weekend_temperature(filename):
   
    weather_data = pandas.read_csv(filename)

    q = """
    SELECT avg(cast (meantempi as integer))
    FROM weather_data
    WHERE cast (strftime('%w', date) as integer) == 0
    OR cast (strftime('%w', date) as integer) == 6
    """
    
    #Execute your SQL command against the pandas frame
    mean_temp_weekends = pandasql.sqldf(q.lower(), locals())
    return mean_temp_weekends

In [9]:
avg_weekend_temperature(weather_data)

,avg(cast (meantempi as integer))
0,65.111111


###Mean Temp on Rainy Days 

In [1]:
def avg_min_temperature(filename):

    weather_data = pandas.read_csv(filename)

    q = """
    SELECT avg(cast (mintempi as integer))
    FROM weather_data['mintempi']
    WHERE rain = 1
    AND cast (mintempi as integer) > 55
    """
    
    #Execute your SQL command against the pandas frame
    avg_min_temp_rainy = pandasql.sqldf(q.lower(), locals())

    return avg_min_temp_rainy

In [11]:
avg_min_temperature(weather_data)

,avg(cast (mintempi as integer))
0,61.25


##Wrangling Subway Data 

###Fixing Turnstile Data 

Taking the data from the MTA website (http://web.mta.info/developers/turnstile.html), the data has numerous data points in each row of the text files:   

In [12]:
import csv
import os

In [13]:
def read_first_line_of_first_textfile(filepath):
    
    for file in os.listdir(filepath):
        if file.endswith(".txt"):

            infile = open(filepath+'/'+file)
            first_line = infile.readline()
            print first_line

            break 

In [14]:
raw_files_path = 'data/turnstile_data/raw_files/'
read_first_line_of_first_textfile(raw_files_path)

IA002,R051,02-00-00,04-17-10,00:00:00,REGULAR,002704717,000928793,04-17-10,04:00:00,REGULAR,002704723,000928795,04-17-10,08:00:00,REGULAR,002704731,000928816,04-17-10,12:00:00,REGULAR,002704835,000928898,04-17-10,16:00:00,REGULAR,002705074,000928943,04-17-10,20:00:00,REGULAR,002705399,000928981,04-18-10,00:00:00,REGULAR,002705426,000928987,04-18-10,04:00:00,REGULAR,002705426,000928987              



In [15]:
# Helper method to aggregate all text files 
def aggregate_text_files_from_path(path):
    
    textfiles = [] 
    
    for file in os.listdir(path):

        if file.endswith(".txt"):
            textfiles.append(path+"/"+file)

    return textfiles

In [16]:
def fix_turnstile_data(filenames):
    
    for name in filenames:
        with open(name) as r:
            read = csv.reader(r)

            parent_path = os.path.split(name)[0]
            grandparent_path = os.path.split(parent_path)[0]
            path = os.path.split(name)[1]
            name = os.path.splitext(path)[0]
            
            newpath = grandparent_path+"/updated_raw_files/" 
            if not os.path.exists(newpath): os.makedirs(newpath) 
                        
            with open(newpath+'updated_'+name+'.txt', 'wb') as f:
                writer = csv.writer(f)
            
                for row in read:                 
                    writer.writerows([row[:3]+row[x:x+5] for x in range(3,len(row),5)])


In [17]:
data_file = os.path.realpath('data/turnstile_data/raw_files/')

filenames = aggregate_text_files_from_path(data_file)

fix_turnstile_data(filenames)

#####Check the first updated file for changes:

In [18]:
updated_file_path = 'data/turnstile_data/updated_raw_files'
read_first_line_of_first_textfile(updated_file_path)



IA002,R051,02-00-00,04-17-10,00:00:00,REGULAR,002704717,000928793



###Combining Turnstile Data 

In [19]:
def create_master_csv_turnstile_file(filenames):
    
    output_file = None 
    # create an output file in the parent directory of the first file 
    for name in filenames:
        with open(name) as r:
            read = csv.reader(r)

            parent_path = os.path.split(name)[0]
            grandparent_path = os.path.split(parent_path)[0]
            path = os.path.split(name)[1]
            name = os.path.splitext(path)[0]
            
            newpath = grandparent_path+"/combined_files/" 
            if not os.path.exists(newpath): os.makedirs(newpath) 
            output_file = newpath+"combined_from_"+name+'.csv'  
            break 
    
    
    with open(output_file, 'w') as master_file:
       master_file.write('C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn\n')

       for filename in filenames:

            with open(filename, 'r') as data_file:
                data_file.next()
                # Gather the remaining data
                for line in data_file:
                    # Save read content to master_file
                    master_file.write(line)
                
    return output_file 

In [20]:
updated_file_path = os.path.realpath('data/turnstile_data/updated_raw_files/')

files_to_combine = aggregate_text_files_from_path(updated_file_path)

combined_file = create_master_csv_turnstile_file(files_to_combine)

#####Check the combined file:

In [21]:

f=open(combined_file)
for line in f:
    print line
    nextline=f.next()
    print "next line", nextline
    break
f.close()

C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn

next line IA002,R051,02-00-00,04-17-10,04:00:00,REGULAR,002704723,000928795



###Filtering Irregular Data 

In [22]:
def filter_by_regular(filename):
    '''
    This function should read the csv file located at filename into a pandas dataframe,
    and filter the dataframe to only rows where the 'DESCn' column has the value 'REGULAR'.
    '''
    
    
    turnstile_data = pandas.read_csv(filename)
    turnstile_data = pandas.DataFrame(turnstile_data)
    return turnstile_data


In [23]:
df_filtered_by_regular = filter_by_regular(combined_file)

#####Check filtered dataframe for changes:

In [24]:
print df_filtered_by_regular.head()

     C/A  UNIT       SCP     DATEn     TIMEn    DESCn  ENTRIESn  EXITSn
0  IA002  R051  02-00-00  04-17-10  04:00:00  REGULAR   2704723  928795
1  IA002  R051  02-00-00  04-17-10  08:00:00  REGULAR   2704731  928816
2  IA002  R051  02-00-00  04-17-10  12:00:00  REGULAR   2704835  928898
3  IA002  R051  02-00-00  04-17-10  16:00:00  REGULAR   2705074  928943
4  IA002  R051  02-00-00  04-17-10  20:00:00  REGULAR   2705399  928981


###Get Hourly Entries 

In [25]:
def get_hourly_entries(df):

    '''
    1) Create a new column called ENTRIESn_hourly
    2) Assign to the column the difference between ENTRIESn of the current row 
    and the previous row. If there is any NaN, fill/replace it with 1.
    '''
 
    df['ENTRIESn_hourly'] = df['ENTRIESn'] - df['ENTRIESn'].shift(1)
    df.fillna(1, inplace=True)    
    return df

In [26]:
df_get_hourly_entries = get_hourly_entries(df_filtered_by_regular)

###Get Hourly Exits 

In [27]:
def get_hourly_exits(df):
    '''
    1) Create a new column called EXITSn_hourly
    2) Assign to the column the difference between EXITSn of the current row 
    and the previous row. If there is any NaN, fill/replace it with 0.
    '''
    
    df['EXITSn_hourly'] = df['EXITSn'] - df['EXITSn'].shift(1)
    df.fillna(0, inplace=True)   
    return df

In [28]:
df_get_hourly_entries_and_exits = get_hourly_exits(df_get_hourly_entries)

#####Check filtered dataframe for changes:

In [29]:
print df_get_hourly_entries_and_exits.head()

     C/A  UNIT       SCP     DATEn     TIMEn    DESCn  ENTRIESn  EXITSn  \
0  IA002  R051  02-00-00  04-17-10  04:00:00  REGULAR   2704723  928795   
1  IA002  R051  02-00-00  04-17-10  08:00:00  REGULAR   2704731  928816   
2  IA002  R051  02-00-00  04-17-10  12:00:00  REGULAR   2704835  928898   
3  IA002  R051  02-00-00  04-17-10  16:00:00  REGULAR   2705074  928943   
4  IA002  R051  02-00-00  04-17-10  20:00:00  REGULAR   2705399  928981   

   ENTRIESn_hourly  EXITSn_hourly  
0                1              0  
1                8             21  
2              104             82  
3              239             45  
4              325             38  


###Time to Hour

In [30]:
def time_to_hour(time):

    
    hour = int(time.split(':')[0])
    return hour

def get_hours(df):

    hours_converted = []
    for hours in df['TIMEn']:

        hours_converted.append(time_to_hour(hours))

    df['HOURSn'] = hours_converted
    return df 

In [31]:
df_get_hours = get_hours(df_get_hourly_entries_and_exits)

###Reformat Subway Dates

In [32]:
import datetime 
def reformat_subway_dates(date):

    date_formatted = str(datetime.datetime.strptime(date, "%m-%d-%y"))[:10]
    return date_formatted

def get_dates(df):

    dates_converted = []

    for date in df['DATEn']:

        dates_converted.append(reformat_subway_dates(date))

    df['DATEn'] = dates_converted

    return df


In [33]:
df_get_dates = get_dates(df_get_hours)

#####Check filtered dataframe for changes:

In [34]:
print df_get_dates.head()

     C/A  UNIT       SCP       DATEn     TIMEn    DESCn  ENTRIESn  EXITSn  \
0  IA002  R051  02-00-00  2010-04-17  04:00:00  REGULAR   2704723  928795   
1  IA002  R051  02-00-00  2010-04-17  08:00:00  REGULAR   2704731  928816   
2  IA002  R051  02-00-00  2010-04-17  12:00:00  REGULAR   2704835  928898   
3  IA002  R051  02-00-00  2010-04-17  16:00:00  REGULAR   2705074  928943   
4  IA002  R051  02-00-00  2010-04-17  20:00:00  REGULAR   2705399  928981   

   ENTRIESn_hourly  EXITSn_hourly  HOURSn  
0                1              0       4  
1                8             21       8  
2              104             82      12  
3              239             45      16  
4              325             38      20  


###Save the combined file as a new csv file

In [35]:
def make_new_path(home, new_path):
    newpath = home+new_path
    if not os.path.exists(newpath): os.makedirs(newpath) 
        
    return newpath


In [36]:
updated_combined_files_path = make_new_path(path, '/turnstile_data/updated_combined_files')

In [37]:
updated_data = df_get_dates
updated_data.to_csv(updated_combined_files_path+'/updated_combined_file.csv')